In [1]:
import pandas as pd

In [2]:
from constants import DATA_DIR, HEADER_TEST_SAMPLE

In [3]:
train_sample = pd.read_csv(DATA_DIR + 'train_sample.tsv', sep='\t', dtype={'account_id': int, 'altered': int, 'create_ts': int,
                                                                          'industry_id': int, 'op_day': int, 'product_class': int,
                                                                          'product_id': int, 'status': int},
                          parse_dates=['op_date'])
train_sample.head()

,account_id,altered,bid,create_ts,crow,exposure_time,industry_id,op_date,op_day,product_class,product_id,sizes,status,ad_id
0,18752,0,83,1543563617,area:7572,"70368475742208,70368475742208,70368475742208,7...",136,2019-02-17,20181130,13,32534,40,1,32
1,18752,0,83,1543563617,area:7572,"70368475742208,70368475742208,70368475742208,7...",136,2019-02-18,20181130,13,32534,40,1,32
2,18752,0,83,1543563617,area:7572,"70368475742208,70368475742208,70368475742208,7...",136,2019-02-19,20181130,13,32534,40,1,32
3,789,1,100,1551916367,area:6410,"4398045462528,4398045462528,4398045462528,4398...",98,2019-03-06,20190306,13,18194,30,1,84
4,6944,1,70,1551253927,"age:217,601,202,837,942,638,287,5,394,347,731,...","281474976694272,281474976694272,28147497669427...",170,2019-02-27,20190227,13,11250,34,1,85


In [4]:
print(len(train_sample))
print(train_sample.altered.sum())
print(train_sample.status.sum())

167168
23776
167168


In [5]:
train_sample.dtypes

account_id                int64
altered                   int64
bid                       int64
create_ts                 int64
crow                     object
exposure_time            object
industry_id               int64
op_date          datetime64[ns]
op_day                    int64
product_class             int64
product_id                int64
sizes                     int64
status                    int64
ad_id                     int64
dtype: object

In [6]:
from datetime import timedelta
gap = timedelta(days=1)
train_sample['next_day'] = pd.to_datetime(train_sample.op_date.dt.date + gap).dt.strftime('%Y%m%d').astype(int)
train_sample.head()

,account_id,altered,bid,create_ts,crow,exposure_time,industry_id,op_date,op_day,product_class,product_id,sizes,status,ad_id,next_day
0,18752,0,83,1543563617,area:7572,"70368475742208,70368475742208,70368475742208,7...",136,2019-02-17,20181130,13,32534,40,1,32,20190218
1,18752,0,83,1543563617,area:7572,"70368475742208,70368475742208,70368475742208,7...",136,2019-02-18,20181130,13,32534,40,1,32,20190219
2,18752,0,83,1543563617,area:7572,"70368475742208,70368475742208,70368475742208,7...",136,2019-02-19,20181130,13,32534,40,1,32,20190220
3,789,1,100,1551916367,area:6410,"4398045462528,4398045462528,4398045462528,4398...",98,2019-03-06,20190306,13,18194,30,1,84,20190307
4,6944,1,70,1551253927,"age:217,601,202,837,942,638,287,5,394,347,731,...","281474976694272,281474976694272,28147497669427...",170,2019-02-27,20190227,13,11250,34,1,85,20190228


In [7]:
label = pd.read_csv(DATA_DIR + 'label.tsv', sep='\t', names=['ad_id', 'next_day', 'label'], header=0) 
dataset = label.merge(train_sample[[col for col in train_sample.columns if col not in ('status', )]])
to_rep_cols = [col for col in HEADER_TEST_SAMPLE if col in dataset.columns]
dataset[to_rep_cols] = dataset[to_rep_cols].replace(-1, 0)  # 重要！否则生成libffm出错
print(len(dataset))
dataset

58554


,ad_id,next_day,label,account_id,altered,bid,create_ts,crow,exposure_time,industry_id,op_date,op_day,product_class,product_id,sizes
0,10003,20190222,2,6539,0,31,1547593423,area:7572,"281474976694272,281474976694272,28147497669427...",186,2019-02-21,20190115,13,9446,30
1,100046,20190309,2,22818,0,150,1546206412,area:3269,"281474976694272,281474976694272,28147497669427...",17,2019-03-08,20181230,13,20433,34
2,100046,20190310,1,22818,0,150,1546206412,area:3269,"281474976694272,281474976694272,28147497669427...",17,2019-03-09,20181230,13,20433,34
3,100046,20190314,1,22818,0,150,1546206412,area:3269,"281474976694272,281474976694272,28147497669427...",17,2019-03-13,20181230,13,20433,34
4,100091,20190219,5,1385,0,31,1546292099,area:7572,"281474976694272,281474976694272,28147497669427...",84,2019-02-18,20181231,13,24175,30
5,100104,20190219,1,239,0,31,1536495666,area:7572,"281474976694272,281474976694272,28147497669427...",17,2019-02-18,20180909,13,15757,30
6,100104,20190222,12,239,1,150,1536495666,area:7572,"281474976694272,281474976694272,28147497669427...",17,2019-02-21,20190221,13,15757,30
7,100104,20190225,4,239,1,133,1536495666,area:7572,"281474976694272,281474976694272,28147497669427...",17,2019-02-24,20190224,13,15757,30
8,100104,20190227,3,239,1,129,1536495666,area:7572,"281474976694272,281474976694272,28147497669427...",17,2019-02-26,20190226,13,15757,30
9,100104,20190228,9,239,0,129,1536495666,area:7572,"281474976694272,281474976694272,28147497669427...",17,2019-02-27,20190226,13,15757,30


In [8]:
dataset.to_hdf(DATA_DIR + 'clean_data.h5', 'dataset')
dataset.to_csv(DATA_DIR + 'dataset.tsv', index=None, sep='\t')

In [9]:
# libffm, 好像id不是从小到大的
id_set = {'sizes', 'industry_id', 'product_class', 'product_id', 'account_id'}
def make_libffm(values, col, idx, id_set):
    if col in id_set:
        # 还要去重
        sorted_int_values = sorted(set((int(v) for v in values.strip(',').split(',')
                                       )))
        return ' '.join(('%s:%s:1' % (idx, v) for v in sorted_int_values))
    if col == 'bid':
        return '%s:0:%s' % (idx, values)
    if col == 'exposure_time':
        bin_values = ''.join([bin(int(x))[2:].zfill(48) for x in values.split(',')])
        return ' '.join(['%s:%s:%s' % (idx, k, v) for k, v in enumerate(bin_values)])
    if col == 'crow':
        pass
    raise Exception('unknow col: %s' % col)

user_fea_idx_plus = {}
user_fea_col = ['age', 'gender', 'area', 'status', 'education', 'consuptionAbility', 'os', 'work', 'connectionType', 'behavior']
user_fea_id_set = set(user_fea_col)
for idx, col in enumerate(user_fea_col):
    user_fea_idx_plus[col] = idx

sep_label_fea=' '
all_crow_v = '|'.join(['%s:0' % col for col in user_fea_col])
# crow字段需要放在最后, 要按照从小到大的顺序来, crow里面也应如此
fea_cols = ['sizes', 'industry_id', 'product_class', 'product_id', 'account_id', 'exposure_time', 'bid', 'crow']

def _get_libffm_line(tup):
    line = str(tup.label)
    # 人群特征需要单独处理
    for idx, col in enumerate(fea_cols):
        if col == 'crow':
            crow = tup.crow
            if crow == 'all':
                crow = all_crow_v
            kvs = {}
            for crow_kv in crow.split('|'):
                crow_col, values = crow_kv.split(':')
                kvs[crow_col] = values
            # 未定义的user_fea为0
            for crow_col in user_fea_col:
                values = kvs.get(crow_col, '0')
                line += ' ' + make_libffm(values, crow_col, idx+user_fea_idx_plus[crow_col], user_fea_id_set)
        else:
            values = str(tup[idx])
            line += ' ' + make_libffm(values, col, idx, id_set)
            
    return line

with open(DATA_DIR + 'data.libffm', 'w') as f:
    for tup in dataset[fea_cols + ['label']].itertuples(index=False):
        f.write('%s\n' % _get_libffm_line(tup))